In [1]:
import warnings

import pandas as pd

from data_processor import DataProcessor
warnings.filterwarnings("ignore")
from utils.logger_config import LoggerCustom
LoggerCustom.set_level("DEBUG")

In [2]:
dp = DataProcessor()
dp.run(
    input_directory="../data/PCAP",
    output_directory="../out",
    append=False
)

[INFO] Logger: <Starting Data processing...>
[INFO] Logger: <Cleared output directory.>
[DEBUG] Logger: <No processed_files.pickle file found. Creating new one...>
[DEBUG] Logger: <Read file: www.youtube.com_94_10.10.57.163.pcap>
[DEBUG] Logger: <Read file: www.youtube.com_12_10.10.57.162.pcap>
[DEBUG] Logger: <Concatenated file: www.youtube.com_12_10.10.57.162.pcap>
[DEBUG] Logger: <Chunk size reached, saving chunk #1>
[DEBUG] Logger: <Read file: ww.google.com.tw_164_10.10.57.164.pcap>
[DEBUG] Logger: <Read file: www.youtube.com_44_10.10.57.163.pcap>
[DEBUG] Logger: <Concatenated file: www.youtube.com_44_10.10.57.163.pcap>
[DEBUG] Logger: <Read file: ww.onclkds.com_154_10.10.57.164.pcap>
[DEBUG] Logger: <Concatenated file: ww.onclkds.com_154_10.10.57.164.pcap>
[DEBUG] Logger: <Read file: www.youtube.com_15_10.10.57.164.pcap>
[DEBUG] Logger: <Concatenated file: www.youtube.com_15_10.10.57.164.pcap>
[DEBUG] Logger: <Chunk size reached, saving chunk #2>
[DEBUG] Logger: <Read file: ww.ins

In [4]:
df = pd.read_feather("../out/chunk_1.ftr")
print(df.head())

            src           dst  dst_port  src_port  proto  size  \
0   10.10.57.38  10.10.57.255       138       138     17   229   
1  10.10.57.113   224.0.0.251      5353      5353     17   425   
2  10.10.57.113   224.0.0.251      5353      5353     17    80   
3  10.10.57.163  10.10.57.255       137       137     17    78   
4  10.10.57.163   224.0.0.252      5355     65336     17    50   

                           time provider  tcp  udp  payload_size  \
0 2019-07-30 17:37:08.512995840  youtube    0    1           229   
1 2019-07-30 17:37:08.512995840  youtube    0    1           425   
2 2019-07-30 17:37:08.614459136  youtube    0    1            80   
3 2019-07-30 17:37:10.065138944  youtube    0    1            78   
4 2019-07-30 17:37:10.065628928  youtube    0    1            50   

   fragmentation  
0              0  
1              0  
2              0  
3              0  
4              0  


In [ ]:
df.shape